In [84]:
import numpy as np
import pandas as pd
import recommenders as rec
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
import math
import timeit
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)
%matplotlib inline

In [85]:
df_city_checkin=pd.read_csv("../data/checkin/lasvegas.csv",converters={'categories':string_to_array})
users_id=df_city_checkin['user_id'].drop_duplicates().reset_index(drop=True)

# USG uses unique checkins cij = 1 or cij = 0
df_city_checkin=df_city_checkin.drop_duplicates(subset=['user_id','business_id']).reset_index(drop=True)

In [88]:
df_city_checkin=df_city_checkin.set_index('user_id')
df_city_checkin.head()

,business_id,date,latitude,longitude,categories,subarea_id
user_id,,,,,,
46TNf-5T5EcNFRJPYv_nrw,ujmEBvifdJM6h6RLv4wQIg,2015-08-19 16:39:53,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
56iEnLi8jR--2ranjPSQ4w,ujmEBvifdJM6h6RLv4wQIg,2013-08-04 10:55:04,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
UDKqdVT0FrpL19shSOqgow,ujmEBvifdJM6h6RLv4wQIg,2017-03-15 23:27:38,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
Nw6Jza_cb0aqd4PT769Vzw,ujmEBvifdJM6h6RLv4wQIg,2013-01-04 22:04:02,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
3b5EsiTkqgdbxPXz7LfBpA,ujmEBvifdJM6h6RLv4wQIg,2014-08-11 23:57:54,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0


# U: preference

$CosineSimilarity = cos(\pmb x, \pmb y) = \frac {\pmb x \cdot \pmb y}{||\pmb x|| \cdot ||\pmb y||}$

In [89]:
def user_similarity(user_checkin_1,user_checkin_2):
    common_checkin=pd.merge(user_checkin_1,user_checkin_2,on='business_id')
    return common_checkin.date_x.count()/(math.sqrt(user_checkin_1.date.count())
                                   *math.sqrt(user_checkin_2.date.count()))

user_similarity(df_city_checkin.loc['46TNf-5T5EcNFRJPYv_nrw'],
                df_city_checkin.loc['56iEnLi8jR--2ranjPSQ4w'])

0.02973505167250263

$\#df\_user\_similarity=\frac{\#usuarios\cdot (\#usuarios-1)}{2}=\sum_{i=1}^{\#usuarios}{i}$

In [ ]:
df_user_similarity=pd.DataFrame(columns=['user_id_1','user_id_2','similarity'])
i=0
start = timeit.default_timer()
# loop sobre os ids dos usuarios
for index_1,user_id_1 in users_id.iteritems():
    # lista de checkins do usuario
    df_user_checkin_1=df_city_checkin.loc[user_id_1]
    # Exclui o proprio usuario
    for index_2,user_id_2 in users_id.drop(index_1).iteritems():
        if len(df_user_similarity[((df_user_similarity.user_id_1==user_id_1) | (df_user_similarity.user_id_1==user_id_2)) &
                        ((df_user_similarity.user_id_2==user_id_1) | (df_user_similarity.user_id_2==user_id_2))])!=0:
            continue
            
        df_user_checkin_2=df_city_checkin.loc[user_id_2]
        sim=user_similarity(df_user_checkin_1,df_user_checkin_2)
        df_user_similarity=df_user_similarity.append({'user_id_1':user_id_1,'user_id_2':user_id_2,'similarity':sim},ignore_index=True)
    if i % 10 == 0:
        print("user",i)
    i=i+1

stop = timeit.default_timer()
print('Time:', stop - start)
df_user_similarity

In [ ]:
def user_cf(user_id,user_checkin,business_id,df_checkin):
    df_business_checkin=df_checkin[df_checkin['business_id']==business_id]
    df_checkin=df_checkin[df_city_checkin['user_id']!=user_id]
    return df_checkin.apply(lambda checkin: user_similarity(user_checkin,df_checkin[df_checkin['user_id']==checkin['user_id']]),axis=1)

user_cf('46TNf-5T5EcNFRJPYv_nrw',df_city_checkin[df_city_checkin.user_id=='46TNf-5T5EcNFRJPYv_nrw'],'ujmEBvifdJM6h6RLv4wQIg',df_city_checkin)

In [40]:
for index,user_id in users_id.iteritems():
    print(user_id)
    df_user_checkin=df_city_checkin[df_city_checkin[.index]==user_id]
    print(df_user_checkin)
    break

46TNf-5T5EcNFRJPYv_nrw
                  user_id             business_id                 date  \
0  46TNf-5T5EcNFRJPYv_nrw  ujmEBvifdJM6h6RLv4wQIg  2015-08-19 16:39:53   

    latitude   longitude                                         categories  \
0  36.215546 -115.248798  [Fitness & Instruction, Doctors, Gyms, Emergen...   

   subarea_id  
0      1415.0  


# S: social

In [61]:
df_city_user=pd.read_csv("../data/user/lasvegas.csv",converters={'friends':string_to_array})

In [62]:
df_city_user.set_index('user_id',inplace=True)
df_city_user.head()

,friends
user_id,
bc8C_eETBWL0olvFSJJd0w,"[4N-HU_T32hLENLntsNKNBg, lwhksSpgIyeYZor_HlN93..."
QiDtS54JwTGeH81c-a84WQ,"[s-591-mtIyP7F1Lffw98jw, Nh-ztLj1IxS8ROHQqDFSi..."
NcUMNz6tAahD6mDmuKwYIA,"[jviCbPOvb08T6nZu5R1tkw, KyMkKb5IohKb0S7v4Jv-o..."
i3dgAM1hWY9UdUCNMDnLXQ,"[pxzs-Dy2hXTis-PuNCV37Q, uZCy7wuptQo3arWvhpqZA..."
U4INQZOPSUaj8hMjLlZ3KA,"[01prx46XR0mMJpNjpGeS6Q, -C-l8EHSLXtZZVfUAUhsP..."


$SI_{k,i}=\eta\cdot\frac{|F_k\cap F_i|}{|F_k\cup F_i|} + (1-\eta)\cdot\frac{|L_k\cap L_i|}{|L_k\cup L_i|}$

$F_k:$ is the friend set of user k

$L_k:$ denote the POI set of user k

$\eta:$ tuning parameter, smaller than 0.05 gives optimal result.(optimal setting)

In [96]:
def social_influence(df_friends_1,df_friends_2,df_user_poi_1,df_user_poi_2,tuning_parameter):
    common_friends=len(pd.merge(df_friends_1,df_friends_2))
    union_friends=len(pd.concat([df_friends_1,df_friends_2]).drop_duplicates())
    common_pois=len(pd.merge(df_user_poi_1,df_user_poi_2))
    union_pois=len(pd.concat([df_user_poi_1,df_user_poi_2]).drop_duplicates())
    return tuning_parameter*common_friends/union_friends+(1-tuning_parameter)*common_pois/union_pois
df_friends_1=df_city_user.loc['56iEnLi8jR--2ranjPSQ4w']
df_friends_2=df_city_user.loc['Nw6Jza_cb0aqd4PT769Vzw']

df_friends_1=pd.DataFrame(df_friends_1['friends'],columns=['user_id'])
df_friends_2=pd.DataFrame(df_friends_2['friends'],columns=['user_id'])
df_user_poi_1=df_city_checkin.loc['56iEnLi8jR--2ranjPSQ4w']['business_id'].drop_duplicates()
df_user_poi_2=df_city_checkin.loc['Nw6Jza_cb0aqd4PT769Vzw']['business_id'].drop_duplicates()


social_influence(df_friends_1,df_friends_2,df_user_poi_1,df_user_poi_2,0.05)

0.02289156626506024

In [135]:
df_user_social_influence=pd.DataFrame(columns=['user_id_1','user_id_2','social_influence'])
num_users=len(users_id)


start = timeit.default_timer()
for index_1,user_id_1 in users_id.iteritems():
    # lista de checkins do usuario
    #df_user_checkin_1=df_city_checkin[df_city_checkin.user_id==user_id_1]
    #df_city_user
    friends_list=df_city_user.loc[user_id_1]['friends']
    df_friends_1=pd.DataFrame(friends_list,columns=['user_id'])
    df_user_poi_1=df_city_checkin.loc[user_id_1]['business_id'].drop_duplicates()
    for user_id_2 in friends_list:
        df_friends_2=pd.DataFrame(df_city_user.loc[user_id_2]['friends'],columns=['user_id'])
        df_user_poi_2=df_city_checkin.loc[user_id_2]['business_id'].drop_duplicates()
        si=social_influence(df_friends_1,df_friends_2,df_user_poi_1,df_user_poi_2,0.05)
        df_user_social_influence=df_user_social_influence.append({'user_id_1':user_id_1,'user_id_2':user_id_2,'social_influence':si},ignore_index=True)
    if index_1%125==0:
        print(str(100*index_1/num_users)+"%")
stop = timeit.default_timer()
print('Time:', stop - start)

0.0%
0.8078849571820973%
1.6157699143641946%
2.423654871546292%
3.231539828728389%
4.039424785910486%
4.847309743092584%
5.655194700274681%
6.463079657456778%
7.270964614638875%
8.078849571820973%
8.88673452900307%
9.694619486185168%
10.502504443367265%
11.310389400549361%
12.118274357731458%
12.926159314913557%
13.734044272095653%
14.54192922927775%
15.349814186459849%
16.157699143641945%
16.965584100824042%
17.77346905800614%
18.581354015188236%
19.389238972370336%
20.197123929552433%
21.00500888673453%
21.812893843916626%
22.620778801098723%
23.42866375828082%
24.236548715462916%
25.044433672645017%
25.852318629827113%
26.66020358700921%
27.468088544191307%
28.275973501373404%
29.0838584585555%
29.8917434157376%
30.699628372919697%
31.507513330101794%
32.31539828728389%
33.12328324446599%
33.931168201648084%
34.73905315883018%
35.54693811601228%
36.354823073194375%
37.16270803037647%
37.97059298755857%
38.77847794474067%
39.58636290192277%
40.394247859104865%
41.20213281628696%
42.0

In [9]:
df_city_user.head()

,user_id,friends
0,bc8C_eETBWL0olvFSJJd0w,"[4N-HU_T32hLENLntsNKNBg, lwhksSpgIyeYZor_HlN93..."
1,QiDtS54JwTGeH81c-a84WQ,"[s-591-mtIyP7F1Lffw98jw, Nh-ztLj1IxS8ROHQqDFSi..."
2,NcUMNz6tAahD6mDmuKwYIA,"[jviCbPOvb08T6nZu5R1tkw, KyMkKb5IohKb0S7v4Jv-o..."
3,i3dgAM1hWY9UdUCNMDnLXQ,"[pxzs-Dy2hXTis-PuNCV37Q, uZCy7wuptQo3arWvhpqZA..."
4,U4INQZOPSUaj8hMjLlZ3KA,"[01prx46XR0mMJpNjpGeS6Q, -C-l8EHSLXtZZVfUAUhsP..."


In [10]:
i=0
df_city_checkin[df_city_checkin.index=='46TNf-5T5EcNFRJPYv_nrw']
for index,checkin in df_city_checkin.iteritems():
    print(index)
    print(checkin)
    if i%22==0:
        print(i)
        break
    i=i+1

business_id
user_id
46TNf-5T5EcNFRJPYv_nrw    ujmEBvifdJM6h6RLv4wQIg
56iEnLi8jR--2ranjPSQ4w    ujmEBvifdJM6h6RLv4wQIg
UDKqdVT0FrpL19shSOqgow    ujmEBvifdJM6h6RLv4wQIg
Nw6Jza_cb0aqd4PT769Vzw    ujmEBvifdJM6h6RLv4wQIg
3b5EsiTkqgdbxPXz7LfBpA    ujmEBvifdJM6h6RLv4wQIg
                                   ...          
i7K3PDFJzXJSebzCj8UYJw    7BIH7ga7YotCM_Bh6qplIA
i7K3PDFJzXJSebzCj8UYJw    7BIH7ga7YotCM_Bh6qplIA
I3E0WQMyDhkvfrLHVot00w    lPgE2MIteQVzs39bwfmS6Q
vWUj8ed8ojO8CctKP-mwxQ    lPgE2MIteQVzs39bwfmS6Q
vWUj8ed8ojO8CctKP-mwxQ    lPgE2MIteQVzs39bwfmS6Q
Name: business_id, Length: 311191, dtype: object
0


# G: geographical



$y'(x',w)=w_0+w_1\cdot x'$